<a href="https://colab.research.google.com/github/jmelinav/Feature-Store-for-Machine-Learning/blob/main/Chapter03/feast_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feast==0.18.1

     |████████████████████████████████| 275 kB 3.2 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 2.3 MB 60.9 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 100 kB 9.8 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 10.9 MB 41.1 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 4.3 MB 52.1 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 402 kB 53.3 MB/s 
     |████████████████████████████████| 111 kB 55.9 MB/s 
     |████████████████████████████████| 3.8 MB 44.7 MB/s 
  Created wheel for pandavro: f

In [2]:
!pip install Pygments

In [3]:
!feast init demo

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage

Creating a new Feast repository in /content/demo.



In [2]:
!ls -R /content/demo/

/content/demo/:
data  example.py  feature_store.yaml  __init__.py

/content/demo/data:
driver_stats.parquet  online_store.db  registry.db


In [5]:
import pandas as pd

df = pd.read_parquet("demo/data/driver_stats.parquet")
df.head(10)

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-02-04 23:00:00+00:00,1005,0.716030,0.473600,62,2022-02-19 23:35:33.298
1,2022-02-05 00:00:00+00:00,1005,0.792346,0.142835,244,2022-02-19 23:35:33.298
2,2022-02-05 01:00:00+00:00,1005,0.426341,0.368232,311,2022-02-19 23:35:33.298
3,2022-02-05 02:00:00+00:00,1005,0.846698,0.579238,842,2022-02-19 23:35:33.298
4,2022-02-05 03:00:00+00:00,1005,0.040044,0.121276,192,2022-02-19 23:35:33.298
5,2022-02-05 04:00:00+00:00,1005,0.793047,0.416169,875,2022-02-19 23:35:33.298
6,2022-02-05 05:00:00+00:00,1005,0.042102,0.647329,926,2022-02-19 23:35:33.298
7,2022-02-05 06:00:00+00:00,1005,0.738018,0.343465,943,2022-02-19 23:35:33.298
8,2022-02-05 07:00:00+00:00,1005,0.849758,0.749403,42,2022-02-19 23:35:33.298
9,2022-02-05 08:00:00+00:00,1005,0.753219,0.021039,670,2022-02-19 23:35:33.298


In [6]:
!pygmentize demo/feature_store.yaml

project: demo
registry: data/registry.db
provider: local
online_store:
    path: data/online_store.db


In [7]:
!pygmentize -f terminal16m demo/example.py

# This is an example feature definition file

from google.protobuf.duration_pb2 import Duration

from feast import Entity, Feature, FeatureView, FileSource, ValueType

# Read data from parquet files. Parquet is convenient for local development mode. For
# production, you can use your favorite DWH, such as BigQuery. See Feast documentation
# for more info.
driver_hourly_stats = FileSource(
    path="/content/demo/data/driver_stats.parquet",
    event_timestamp_column="event_timestamp",
    created_timestamp_column="created",
)

# Define an entity for the driver. You can think of entity as a primary key used to
# fetch features.
driver = Entity(name="driver_id", value_type=ValueType.INT64, description="driver id",)

# Our parquet files contain sample data that includes a driver_id column, timestamps and
# three feature column. Here we define a Feature View that will allow us to serve this
# data to our model online.
driver_hourly_stats_view = FeatureView(
    name="driver_hourly_stats",


In [3]:
%cd demo
!feast apply

/content/demo
No changes to registry
No changes to infrastructure


In [4]:
from feast import FeatureStore
store = FeatureStore(repo_path=".")

In [14]:
for entity in store.list_entities():
  print(entity.to_dict())

store.list_feature_views

{'spec': {'name': 'driver_id', 'valueType': 'INT64', 'description': 'driver id', 'joinKey': 'driver_id'}, 'meta': {'createdTimestamp': '2022-02-19T23:57:42.772948Z', 'lastUpdatedTimestamp': '2022-02-19T23:57:42.772948Z'}}


In [18]:
import pandas as pd
import numpy as np
from pytz import timezone, utc
from datetime import datetime, timedelta
import random
days = [datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0).replace(tzinfo=utc) \
        - timedelta(day) for day in range(10)][::-1]

customers = [1001, 1002, 1003, 1004, 1005]
customer_features = pd.DataFrame(
    {
        "datetime": [day for day in days for customer in customers], # Datetime is required
        "customer_id": [customer for day in days for customer in customers], # Customer is the entity
        "daily_transactions": [np.random.rand() * 10 for _ in range(len(days) * len(customers))], # Feature 1
        "total_transactions": [np.random.randint(100) for _ in range(len(days) * len(customers))], # Feature 2
    }
)
customer_features.to_parquet("/content/demo/data/customer_features.parquet")
customer_features.head(5)

,datetime,customer_id,daily_transactions,total_transactions
0,2022-02-11 00:00:00+00:00,1001,2.398850,44
1,2022-02-11 00:00:00+00:00,1002,6.776920,29
2,2022-02-11 00:00:00+00:00,1003,5.944877,73
3,2022-02-11 00:00:00+00:00,1004,2.245862,27
4,2022-02-11 00:00:00+00:00,1005,0.839079,82


In [19]:
!feast apply

Created entity customer_id
Created feature view customer_features

Created sqlite table demo_customer_features



In [24]:
from datetime import datetime, timedelta
import pandas as pd

from feast import FeatureStore

# The entity dataframe is the dataframe we want to enrich with feature values
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003],
        "event_timestamp": [
            datetime.now() - timedelta(minutes=11),
            datetime.now() - timedelta(minutes=36),
            datetime.now() - timedelta(minutes=73),
        ],
    }
)
entity_df.head()

,driver_id,event_timestamp
0,1001,2022-02-20 01:37:36.523322
1,1002,2022-02-20 01:12:36.523340
2,1003,2022-02-20 00:35:36.523343


In [27]:
store = FeatureStore(repo_path=".")

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
).to_df()

training_df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips
0,2022-02-20 00:35:36.523343+00:00,1003,0.459515,0.242431,363
1,2022-02-20 01:12:36.523340+00:00,1002,0.241556,0.996971,575
2,2022-02-20 01:37:36.523322+00:00,1001,0.049006,0.739369,991


In [28]:
!feast materialize-incremental {datetime.now().isoformat()}

Materializing 2 feature views to 2022-02-20 02:13:03+00:00 into the sqlite online store.

driver_hourly_stats from 2022-02-19 02:13:05+00:00 to 2022-02-20 02:13:03+00:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 375.83it/s]
customer_features from 2022-02-19 02:13:05+00:00 to 2022-02-20 02:13:03+00:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 444.15it/s]


In [31]:
store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

feature_vector

{'avg_daily_trips': [34, 256],
 'conv_rate': [0.9326972365379333, 0.07134518772363663],
 'driver_id': [1004, 1005]}